In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
from rasterio.transform import from_origin
import geopandas as gpd
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
import glob


In [2]:
# Read the Parquet file
df = pd.read_parquet("wheat_pixel.parquet")


In [3]:
df

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12
0,30.240077,77.714064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.240077,77.714154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.240077,77.714243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.240077,77.714333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.240077,77.714423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50315695,29.580804,78.329410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315696,29.580804,78.329500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315697,29.580804,78.329589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315698,29.580804,78.329679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data1=df.drop(['timestep_1'],axis = 1)
data1

,latitude,longitude,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12
0,30.240077,77.714064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.240077,77.714154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.240077,77.714243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.240077,77.714333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.240077,77.714423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50315695,29.580804,78.329410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315696,29.580804,78.329500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315697,29.580804,78.329589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315698,29.580804,78.329679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data1 = data1.replace(0, pd.NA)
data = data1.dropna()
data

,latitude,longitude,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12
2689,30.240077,77.955621,0.464052,0.346939,0.277628,0.372645,0.473451,0.391011,0.373134,0.285132,0.284232,0.471639,0.475601
2690,30.240077,77.955711,0.418829,0.280415,0.249036,0.299117,0.424584,0.350165,0.373134,0.277027,0.32809,0.489911,0.482625
2692,30.240077,77.955890,0.309402,0.243094,0.263441,0.238009,0.457127,0.429134,0.417103,0.497133,0.491459,0.588453,0.572169
2693,30.240077,77.955980,0.306991,0.280341,0.222857,0.170553,0.446903,0.329446,0.411824,0.397693,0.477938,0.498465,0.440055
9547,30.239987,77.955890,0.295259,0.244195,0.248848,0.402653,0.456009,0.446719,0.462719,0.522131,0.525719,0.568505,0.541297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.676768,0.710564,0.591766,0.514581,0.542471,0.459556,0.534407,0.406909,0.505331,0.441652,0.192093
50298002,29.580983,77.971611,0.682883,0.711809,0.604716,0.5234,0.533659,0.448927,0.510125,0.381468,0.402174,0.256276,0.259951
50298014,29.580983,77.972689,0.603329,0.626122,0.503268,0.130435,0.197442,0.199153,0.201521,0.19181,0.201313,0.294649,0.17445
50304784,29.580893,77.965053,0.702612,0.69749,0.573227,0.50365,0.4785,0.549342,0.517213,0.474889,0.54259,0.461412,0.464832


In [6]:
import pandas as pd

# List of columns to convert to numerical types
columns_to_convert = [
    'timestep_2', 'timestep_3', 'timestep_4', 'timestep_5', 
    'timestep_6', 'timestep_7', 'timestep_8', 'timestep_9', 'timestep_10', 
    'timestep_11', 'timestep_12'
]

# Convert the specified columns to numerical types
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Create a new DataFrame with the updated values
new_df = data.copy()

# Display the new DataFrame
print(new_df.head())


C:\Users\admin\AppData\Local\Temp\ipykernel_1392\1448844404.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')


       latitude  longitude  timestep_2  timestep_3  timestep_4  timestep_5  \
2689  30.240077  77.955621    0.464052    0.346939    0.277628    0.372645   
2690  30.240077  77.955711    0.418829    0.280415    0.249036    0.299117   
2692  30.240077  77.955890    0.309402    0.243094    0.263441    0.238009   
2693  30.240077  77.955980    0.306991    0.280341    0.222857    0.170553   
9547  30.239987  77.955890    0.295259    0.244195    0.248848    0.402653   

      timestep_6  timestep_7  timestep_8  timestep_9  timestep_10  \
2689    0.473451    0.391011    0.373134    0.285132     0.284232   
2690    0.424584    0.350165    0.373134    0.277027     0.328090   
2692    0.457127    0.429134    0.417103    0.497133     0.491459   
2693    0.446903    0.329446    0.411824    0.397693     0.477938   
9547    0.456009    0.446719    0.462719    0.522131     0.525719   

      timestep_11  timestep_12  
2689     0.471639     0.475601  
2690     0.489911     0.482625  
2692     0.588453

In [7]:
new_df

,latitude,longitude,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12
2689,30.240077,77.955621,0.464052,0.346939,0.277628,0.372645,0.473451,0.391011,0.373134,0.285132,0.284232,0.471639,0.475601
2690,30.240077,77.955711,0.418829,0.280415,0.249036,0.299117,0.424584,0.350165,0.373134,0.277027,0.328090,0.489911,0.482625
2692,30.240077,77.955890,0.309402,0.243094,0.263441,0.238009,0.457127,0.429134,0.417103,0.497133,0.491459,0.588453,0.572169
2693,30.240077,77.955980,0.306991,0.280341,0.222857,0.170553,0.446903,0.329446,0.411824,0.397693,0.477938,0.498465,0.440055
9547,30.239987,77.955890,0.295259,0.244195,0.248848,0.402653,0.456009,0.446719,0.462719,0.522131,0.525719,0.568505,0.541297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.676768,0.710564,0.591766,0.514581,0.542471,0.459556,0.534407,0.406909,0.505331,0.441652,0.192093
50298002,29.580983,77.971611,0.682883,0.711809,0.604716,0.523400,0.533659,0.448927,0.510125,0.381468,0.402174,0.256276,0.259951
50298014,29.580983,77.972689,0.603329,0.626122,0.503268,0.130435,0.197442,0.199153,0.201521,0.191810,0.201313,0.294649,0.174450
50304784,29.580893,77.965053,0.702612,0.697490,0.573227,0.503650,0.478500,0.549342,0.517213,0.474889,0.542590,0.461412,0.464832


In [8]:
rename_dict = {f'timestep_{i}': f'Band_{i}' for i in range(1, 16)}

# Rename the columns
new_df.rename(columns=rename_dict, inplace=True)

In [9]:
Range = new_df[(new_df.Band_2 >= 0) & (new_df.Band_2 <= 1) 
            & (new_df.Band_3 >= 0) & (new_df.Band_3 <= 1)
            & (new_df.Band_4 >= 0) & (new_df.Band_4 <= 1)
            & (new_df.Band_5 >= 0) & (new_df.Band_5 <= 1)
            & (new_df.Band_6 >= 0) & (new_df.Band_6 <= 1)
            & (new_df.Band_7 >= 0) & (new_df.Band_7 <= 1)
            & (new_df.Band_8 >= 0) & (new_df.Band_8 <= 1)
            & (new_df.Band_9 >= 0) & (new_df.Band_9 <= 1)
            & (new_df.Band_10 >= 0) & (new_df.Band_10 <= 1)
            & (new_df.Band_11 >= 0) & (new_df.Band_11 <= 1)
            & (new_df.Band_12 >= 0) & (new_df.Band_12 <= 1)
            
           ]

In [10]:
Range

,latitude,longitude,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12
2689,30.240077,77.955621,0.464052,0.346939,0.277628,0.372645,0.473451,0.391011,0.373134,0.285132,0.284232,0.471639,0.475601
2690,30.240077,77.955711,0.418829,0.280415,0.249036,0.299117,0.424584,0.350165,0.373134,0.277027,0.328090,0.489911,0.482625
2692,30.240077,77.955890,0.309402,0.243094,0.263441,0.238009,0.457127,0.429134,0.417103,0.497133,0.491459,0.588453,0.572169
2693,30.240077,77.955980,0.306991,0.280341,0.222857,0.170553,0.446903,0.329446,0.411824,0.397693,0.477938,0.498465,0.440055
9547,30.239987,77.955890,0.295259,0.244195,0.248848,0.402653,0.456009,0.446719,0.462719,0.522131,0.525719,0.568505,0.541297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.676768,0.710564,0.591766,0.514581,0.542471,0.459556,0.534407,0.406909,0.505331,0.441652,0.192093
50298002,29.580983,77.971611,0.682883,0.711809,0.604716,0.523400,0.533659,0.448927,0.510125,0.381468,0.402174,0.256276,0.259951
50298014,29.580983,77.972689,0.603329,0.626122,0.503268,0.130435,0.197442,0.199153,0.201521,0.191810,0.201313,0.294649,0.174450
50304784,29.580893,77.965053,0.702612,0.697490,0.573227,0.503650,0.478500,0.549342,0.517213,0.474889,0.542590,0.461412,0.464832


In [11]:
wheat = Range.drop(['longitude','latitude'],axis=1)
wheat

,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12
2689,0.464052,0.346939,0.277628,0.372645,0.473451,0.391011,0.373134,0.285132,0.284232,0.471639,0.475601
2690,0.418829,0.280415,0.249036,0.299117,0.424584,0.350165,0.373134,0.277027,0.328090,0.489911,0.482625
2692,0.309402,0.243094,0.263441,0.238009,0.457127,0.429134,0.417103,0.497133,0.491459,0.588453,0.572169
2693,0.306991,0.280341,0.222857,0.170553,0.446903,0.329446,0.411824,0.397693,0.477938,0.498465,0.440055
9547,0.295259,0.244195,0.248848,0.402653,0.456009,0.446719,0.462719,0.522131,0.525719,0.568505,0.541297
...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.676768,0.710564,0.591766,0.514581,0.542471,0.459556,0.534407,0.406909,0.505331,0.441652,0.192093
50298002,0.682883,0.711809,0.604716,0.523400,0.533659,0.448927,0.510125,0.381468,0.402174,0.256276,0.259951
50298014,0.603329,0.626122,0.503268,0.130435,0.197442,0.199153,0.201521,0.191810,0.201313,0.294649,0.174450
50304784,0.702612,0.697490,0.573227,0.503650,0.478500,0.549342,0.517213,0.474889,0.542590,0.461412,0.464832


In [12]:
new_column_names = [19, 24, 34, 84, 100, 110, 115, 128, 133, 159, 164]

# Rename columns
wheat.columns = new_column_names

In [13]:
wheat

,19,24,34,84,100,110,115,128,133,159,164
2689,0.464052,0.346939,0.277628,0.372645,0.473451,0.391011,0.373134,0.285132,0.284232,0.471639,0.475601
2690,0.418829,0.280415,0.249036,0.299117,0.424584,0.350165,0.373134,0.277027,0.328090,0.489911,0.482625
2692,0.309402,0.243094,0.263441,0.238009,0.457127,0.429134,0.417103,0.497133,0.491459,0.588453,0.572169
2693,0.306991,0.280341,0.222857,0.170553,0.446903,0.329446,0.411824,0.397693,0.477938,0.498465,0.440055
9547,0.295259,0.244195,0.248848,0.402653,0.456009,0.446719,0.462719,0.522131,0.525719,0.568505,0.541297
...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.676768,0.710564,0.591766,0.514581,0.542471,0.459556,0.534407,0.406909,0.505331,0.441652,0.192093
50298002,0.682883,0.711809,0.604716,0.523400,0.533659,0.448927,0.510125,0.381468,0.402174,0.256276,0.259951
50298014,0.603329,0.626122,0.503268,0.130435,0.197442,0.199153,0.201521,0.191810,0.201313,0.294649,0.174450
50304784,0.702612,0.697490,0.573227,0.503650,0.478500,0.549342,0.517213,0.474889,0.542590,0.461412,0.464832


## maturity dates 

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, argrelextrema

# Double Gaussian function
def double_gaussian(x, a1, b1, c1, a2, b2, c2):
    return (a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) +
            a2 * np.exp(-(x - b2)**2 / (2 * c2**2)))

# First derivative function
def first_derivative(x, a1, b1, c1, a2, b2, c2):
    return ((-(x - b1) / c1**2) * a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) +
            (-(x - b2) / c2**2) * a2 * np.exp(-(x - b2)**2 / (2 * c2**2)))


days = np.array(wheat.columns, dtype=float)  # Convert column headers to float
data = wheat.values.astype(float)  # Convert the data values to float

# Normalize the data
data_norm = (data - np.min(data, axis=1, keepdims=True)) / (np.max(data, axis=1, keepdims=True) - np.min(data, axis=1, keepdims=True))

# Fit the double Gaussian to each row
fitted_params = []
minima_days = []
for i in range(data_norm.shape[0]):
    try:
        # Adjust initial guesses based on data characteristics
        a1_guess = max(data_norm[i])
        b1_guess = days[np.argmax(data_norm[i])]
        c1_guess = (days[-1] - days[0]) / 10  # Rough estimate for the width of the first peak
        a2_guess = max(data_norm[i]) / 2
        b2_guess = days[len(days) // 2]
        c2_guess = (days[-1] - days[0]) / 5  # Rough estimate for the width of the second peak

        bounds = (0, [np.inf, np.max(days), np.inf, np.inf, np.max(days), np.inf])
        popt, _ = curve_fit(double_gaussian, days, data_norm[i], p0=[a1_guess, b1_guess, c1_guess, a2_guess, b2_guess, c2_guess], bounds=bounds, maxfev=10000)
        fitted_params.append(popt)
        
        # Calculate the first derivative
        first_deriv = first_derivative(days, *popt)
        
        # Find local minima of the first derivative
        minima_indices = argrelextrema(first_deriv, np.less)[0]  # Using numpy's argrelextrema to find indices of local minima
        
        if len(minima_indices) > 0:
            last_minima_day = days[minima_indices[-1]]  # Get the day where the last local minima of the first derivative occurs
        else:
            last_minima_day = np.nan
        minima_days.append(last_minima_day)
        
    except RuntimeError as e:
        # print(f"Fit did not converge for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        minima_days.append(np.nan)
    except Exception as e:
        print(f"An error occurred for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        minima_days.append(np.nan)

# Save minima_days to an Excel file
minima_df = pd.DataFrame({'Row': np.arange(1, len(minima_days) + 1), 'Last_Minima_Day': minima_days})

minima_df.to_parquet('maturity.parquet', index = False)
print("file saved as parquet file")


file saved as parquet file
